In [82]:
def mtf_encode(text):  
    # Все встречающиеся символы в строке
    letters = [str(i) for i in range(256)]
    letters.append(' ')
    letters.append('$')
    # Итоговый результат 
    res = ''  
    # Пройти по всем символам строки 
    for i in range(len(text)):  
        # Определяем номер символа в нашем словаре
        mtf = letters.index(text[i])
        # Эти условия нужны чтобы было легче раскодировать результат алгоритма, 
        # мы заменим не относящиеся к кодированию символы $ и ' ' другими более простыми
        if mtf<256:  
            res += str(mtf)  
        # Если это символ $
        elif mtf == 256:
            res += 'A'
        # Если это символ ' '
        elif mtf == 257:
            res += 'B' 
        # Сдвинем наш словарь символов в соответствии с алгоритмом
        letters = letters[mtf:]+letters[:mtf]
    # Вернем итоговый результат кодирования 
    return res

def mtf_decode(text):
    # Определим такой же словарь символов как и в кодировании
    letters = [str(i) for i in range(256)]
    letters.append(' ')
    letters.append('$')
    # Итоговый результат
    res = ''  
    # Для каждого символа в принимаемой строке
    for char in text:  
        # Проверим, служебный ли это символ и если так найдем его индекс словаре
        if char == 'A':  
            mtf = 256  
        elif char == 'B':  
            mtf = 257  
        # Заменим каждую цифру соответствующим ей символом в словаре
        else:  
            mtf = int(char)  
        res += letters[mtf] 
        # Сдвинем наш словарь в соответствии с алгоритмом
        letters = letters[mtf:]+letters[:mtf]
    return res  

In [84]:
class node:  
    letter = None  
    value = 0  
    left = None  
    right = None  
      
    def __lt__(self,other):  
        return self.value<other.value  
      
def dfs(n,dicti,code = ''):  
    if n.letter:  
        dicti[n.letter] = code  
    if n.left:  
        dfs(n.left, dicti, code+'0')  
    if n.right:  
        dfs(n.right, dicti, code+'1')  
        
def haffman_compression(string):  
    if len(string) % 2 != 0:  
        string+='%'  
    letters = {}  
  
    for i in range(len(string)//2):  
        letter = string[i*2:i*2+2]  
        if letter not in letters:  
            letters[letter] = 1  
        else:  
            letters[letter] += 1  
 
 
    node_list = []  
  
    for key,value in letters.items():  
        n = node()  
        n.letter = key  
        n.value = value  
        node_list.append(n)  
     
    while len(node_list) > 1:  
        node_list.sort(reverse = True)  
        n1 = node_list.pop()  
        n2 = node_list.pop()  
        n3 = node()  
        n3.value = n1.value+n2.value  
        n3.left = n1  
        n3.right = n2  
        node_list.append(n3)  
   
    haffman = {}  
  
    dfs(node_list[0],haffman)  
  
    compressed_string = ''  
  
    for i in range(len(string)//2):  
        letter = string[i*2:i*2+2]  
        compressed_string += haffman[letter]  
      
    return compressed_string, haffman

In [34]:
from collections import Counter

class Node:

    def __init__(self, value, left=None, right=None):
        self.right = right
        self.left = left
        self.value = value


def get_code(root, codes=dict(), code=''):

    if root is None:
        return

    if isinstance(root.value, str):
        codes[root.value] = code
        return codes

    get_code(root.left, codes, code + '0')
    get_code(root.right, codes, code + '1')

    return codes


def get_tree(string):
    string_count = Counter(string)

    if len(string_count) <= 1:
        node = Node(None)

        if len(string_count) == 1:
            node.left = Node([key for key in string_count][0])
            node.right = Node(None)

        string_count = {node: 1}

    while len(string_count) != 1:
        node = Node(None)
        spam = string_count.most_common()[:-3:-1]

        if isinstance(spam[0][0], str):
            node.left = Node(spam[0][0])

        else:
            node.left = spam[0][0]

        if isinstance(spam[1][0], str):
            node.right = Node(spam[1][0])

        else:
            node.right = spam[1][0]

        del string_count[spam[0][0]]
        del string_count[spam[1][0]]
        string_count[node] = spam[0][1] + spam[1][1]

    return [key for key in string_count][0]


def coding(string, codes):
    res = ''

    for symbol in string:
        res += codes[symbol]

    return res


def decoding(string, codes):
    
    codes2={}
    for key,value in codes.items():
        codes2[value] = key
    
    code = ''
    res = ''
    for c in string:
        code = code + c
        if code in codes2:
            res = res+codes2[code]
            code = ''
    return res

In [ ]:
haffman_compression

In [35]:
def BWT(sequence):
    # Добавим символ конца строки 
    sequence += '$'
    # Получим список всех перестановок
    table = [sequence[index:] + sequence[:index] for index, _ in enumerate(sequence)]
    # Отсортируем этот список
    table.sort()
    # Возьмем последнюю букву строки из каждого элемента
    bwt = [rotation[-1] for rotation in table]
    # Это и есть результат кодирования алгоритмом bwt
    bwt = ''.join(bwt)
    return bwt

def inverse_BWT(sequence):
    # Разобьем строку на буквы
    table = [col for col in sequence]
    # Составим таблицу пройдя по всем элементам изначальной строки (первый элемент уже есть в нашей таблице)
    for i in range(len(sequence) - 1):
        # Сортируем элементы в таблице
        table.sort()
        # Склеиваем i-й элемент в таблице с i-м элементом в исходной строке
        table = [sequence[i] + table[i] for i in range(len(sequence))]
    # Результатом декодирования будет элемент, который заканчивается на символ конца строки
    return table[[row[-1] for row in table].index('$')] 

In [36]:
def compress(string):
    
    bwt = BWT(string)
    mtf = mtf_encode(bwt)
    count_symbols = len(mtf)
    
    def get_code(root, codes=dict(), code=''):

        if root is None:
            return

        if isinstance(root.value, str):
            codes[root.value] = code
            return codes

        get_code(root.left, codes, code + '0')
        get_code(root.right, codes, code + '1')

        return codes
    
    codes = get_code(get_tree(mtf))
    coding_str = coding(mtf, codes)
    
    return coding_str, codes, count_symbols

In [37]:
def decompress(string, code):
    
    hoff = decoding(string, code)
    mtf = mtf_decode(hoff)
    res = inverse_BWT(mtf)
    
    return res

In [92]:
def zip_file(path, path_exit, L = 100):
    import os
    
    if not os.path.isfile(path):
        raise Exception('Файл не найден!')
        
    if not os.path.isdir(path_exit):
        raise Exception('Укажите путь для выгрузки результаты работы архиватора!')
    
    f = open(path, 'rb')
    rd = f.read()
    fd = [str(int(char)) for char in rd]  
    f.close()
    
    n_full_blocks = len(fd) // L

    array = []
    for i in range(n_full_blocks):
        array.append(' '.join(fd[i * L : (i + 1) * L]))
        
    array.append(' '.join(fd[n_full_blocks * L : ]))
    
    compressed = []
    keys = []
    counts = []
    for s in array:
        c, k, count = compress(s)
        compressed.append(c)
        keys.append(k)
        counts.append(count)
    
    import pickle
    
    data = {'keys' : keys, 
           'counts' : counts}
    
    os.mkdir(path_exit + r'\zipped')
    with open(path_exit + r'\zipped' + r'\keys.pickle', 'wb') as f:
        pickle.dump(data, f)
    
    compressed = ''.join(compressed)
    print(compressed)
    compressed += '0' * ((len(compressed) // 8 + 1) * 8 - len(compressed))
    
    byte_array = [int(compressed[i * 8 : (i+1) * 8], 2) for i in range (int(len(compressed) / 8))]  
      
    f = open(path_exit + r'\zipped'+ r'\zip.txt', 'wb')
    
    for byte in byte_array:
        f.write(byte.to_bytes(1,'big'))
        
    return 'Completed!'

In [97]:
zip_file(r'C:\Users\Денис\Desktop\Rainier.bmp', r'C:\Users\Денис\Desktop', 1000)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



'Completed!'

In [85]:
def zip_file(path, path_exit, L = 100):
    import os
    
    if not os.path.isfile(path):
        raise Exception('Файл не найден!')
        
    if not os.path.isdir(path_exit):
        raise Exception('Укажите путь для выгрузки результаты работы архиватора!')
    
    f = open(path, 'rb')
    rd = f.read()
    fd = [str(int(char)) for char in rd]  
    f.close()
    
    n_full_blocks = len(fd) // L

    array = []
    for i in range(n_full_blocks):
        array.append(' '.join(fd[i * L : (i + 1) * L]))
        
    array.append(' '.join(fd[n_full_blocks * L : ]))
    
    compressed = []
    keys = []
    for s in array:
        c, k, count = compress(s)
        compressed.append(c)
        keys.append(k)
    
    import pickle
    
    data = {'compressed' : compressed, 
           'keys' : keys}
    
    with open(path_exit + r'\keys.pickle', 'wb') as f:
        pickle.dump(data, f)

    return 'Completed!'

In [41]:
def unzip_file(path, path_exit):
    
    import pickle
    import os
    
    if not os.path.isfile(path):
        raise Exception('Файл не найден!')
        
    if not os.path.isdir(path_exit):
        raise Exception('Укажите путь для выгрузки результаты работы разархиватора!')
        
    with open(path, 'rb') as f:
        data_new = pickle.load(f)
        
    compressed = data_new['compressed']
    keys = data_new['keys']
    
    array = []
    for c, k in zip(compressed, keys):
        text = decompress(c, k)
        array.append(text)
    
    res = []
    for a in array:
        for i in a.split(' '):
            res += i
    
    f = open(path_exit.bmp + r'\result.jpg', 'wb')
    for char in fd_int:
        f.write(char.to_bytes(1,'big')) # преобразование int в byte (1 байт на символ - utf8)

    f.close()
    return 'Completed!'

In [42]:
unzip_file(r'C:\Users\Денис\Desktop\keys.pickle', r'C:\Users\Денис\Desktop')

KeyboardInterrupt: 

In [24]:
example = compressed[0]
example_key = keys[0]

In [23]:
    import pickle
    with open(r'C:\Users\Денис\Desktop\keys.pickle', 'rb') as f:
        data_new = pickle.load(f)
        
    compressed = data_new['compressed']
    keys = data_new['keys']

In [25]:
example

'011110101110011010001110011001111111111111111111111111111111111111111111111111111111111111111111111000000001111111111111000100101111111111111100110100111111111000000001111111111111111111011110110101100000000001111111111111111111111111111111111110111101110001001111010110111101011011110111010010110101101111100110100111110011010000110100111100010010110011010001010011011110110110101011111111100010010111111101110011001111111111111000100101111111111000000001011100110011101010001010111100100101001100100100001010111011000101101000110101000110110010100011010100110001001011100111101101010100010000011011010101000001011100111110101101100101011010010010000110001001000101000111100100111010111001100011000111001111100101000100101010110010001101100010110111110000011011000110100000111010110111110010111101110010001000001000110010001001011100010110000010010111110101001101111111111100010010000100101101111010100010001010000100001110001001011111000000001111000100101111111001100100010001101001100110100111001

In [26]:
example_key

{'6': '0000',
 '1': '0001',
 'B': '0010',
 '2': '0011',
 'A': '0100',
 '5': '01010',
 '3': '01011',
 '8': '01100',
 '7': '01101',
 '4': '01110',
 '9': '01111',
 '0': '1'}

In [27]:
decompress(example, example_key)

'255 216 255 225 3 60 69 120 105 102 0 0 73 73 42 0 8 0 0 0 10 0 15 1 2 0 6 0 0 0 134 0 0 0 16 1 2 0 16 0 0 0 140 0 0 0 26 1 5 0 1 0 0 0 156 0 0 0 27 1 5 0 1 0 0 0 164 0 0 0 40 1 3 0 1 0 0 0 2 0 0 0 49 1 2 0 42 0 0 0 172 0 0 0 50 1 2 0 20 0 0 0 214 0 0 0 59 1 2 0 12 0 0 0 234 0 0 0 152 130 2 0 13 0 0 0 246 0 0 0 105 135 4 0 1 0 0 0 4 1 0 0 0 0 0 0 67 97 110 111 110 0 67 97 110 111 110 32 69 79 83 32 53 68 83 32 82 0 44 1 0 0 1 0 0 0 44 1 0 0 1 0 0 0 65 100 111 98 101 32 80 104 111 116 111 115 104 111 112 32 76 105 103 104 116 114 111 111 109 32 54 46 53 46 49 32 40 87 105 110 100 111 119 115 41 0 50 48 49 54 58 49 49 58 49 56 32 49 50 58 48 49 58 48 53 0 68 105 101 103 111 32 68 101 108 115 111 0 67 67 45 66 89 45 83 65 32 52 46 48 0 0 30 0 154 130 5 0 1 0 0 0 114 2 0 0 157 130 5 0 1 0 0 0 122 2 0 0 34 136 3 0 1 0 0 0 1 0 0 0 39 136 3 0 1 0 0 0 100 0 0 0 48 136 3 0 1 0 0 0 2 0 0 0 50 136 4 0 1 0 0 0 100 0 0 0 0 144 7 0 4 0 0 0 48 50 51 48 3 144 2 0 20 0 0 0 130 2 0 0 4 144 2 0 20 0 0 0

In [30]:
    import os
    
    f = open(r'C:\Users\Денис\Desktop\Monasterio_Khor_Virap,_Armenia,_2016-10-01,_DD_25.jpg', 'rb')
    rd = f.read()
    fd = [str(int(char)) for char in rd]  
    f.close()
    
    n_full_blocks = len(fd) // 2000

    array = []
    for i in range(n_full_blocks):
        array.append(' '.join(fd[i * 2000 : (i + 1) * 2000]))
        
    array.append(' '.join(fd[n_full_blocks * 2000 : ]))

In [32]:
array[0]

'255 216 255 225 3 60 69 120 105 102 0 0 73 73 42 0 8 0 0 0 10 0 15 1 2 0 6 0 0 0 134 0 0 0 16 1 2 0 16 0 0 0 140 0 0 0 26 1 5 0 1 0 0 0 156 0 0 0 27 1 5 0 1 0 0 0 164 0 0 0 40 1 3 0 1 0 0 0 2 0 0 0 49 1 2 0 42 0 0 0 172 0 0 0 50 1 2 0 20 0 0 0 214 0 0 0 59 1 2 0 12 0 0 0 234 0 0 0 152 130 2 0 13 0 0 0 246 0 0 0 105 135 4 0 1 0 0 0 4 1 0 0 0 0 0 0 67 97 110 111 110 0 67 97 110 111 110 32 69 79 83 32 53 68 83 32 82 0 44 1 0 0 1 0 0 0 44 1 0 0 1 0 0 0 65 100 111 98 101 32 80 104 111 116 111 115 104 111 112 32 76 105 103 104 116 114 111 111 109 32 54 46 53 46 49 32 40 87 105 110 100 111 119 115 41 0 50 48 49 54 58 49 49 58 49 56 32 49 50 58 48 49 58 48 53 0 68 105 101 103 111 32 68 101 108 115 111 0 67 67 45 66 89 45 83 65 32 52 46 48 0 0 30 0 154 130 5 0 1 0 0 0 114 2 0 0 157 130 5 0 1 0 0 0 122 2 0 0 34 136 3 0 1 0 0 0 1 0 0 0 39 136 3 0 1 0 0 0 100 0 0 0 48 136 3 0 1 0 0 0 2 0 0 0 50 136 4 0 1 0 0 0 100 0 0 0 0 144 7 0 4 0 0 0 48 50 51 48 3 144 2 0 20 0 0 0 130 2 0 0 4 144 2 0 20 0 0 0

In [54]:
class node:
    code = None
    value = 0
    left = None
    right = None
    def __lt__(self, other):
        self.value < other.value

In [55]:
n1 = node()
n2 = node()
n3 = node()
n4 = node()

In [56]:
n1.code = 'j'
n1.value = 200
n2.code = 'k'
n2.value = 100
n3.code = 'u'
n3.value = 95
n4.code = 'v'
n4.value = 15

In [57]:
n5 = node()
n5.value = n3.value + n4.value
n5.left = n3
n5.right = n4

In [58]:
n6 = node()
n6.value = n2.value + n5.value
n6.left = n2
n6.right = n5

In [59]:
n7 = node()
n7.value = n1.value + n6.value
n7.left = n1
n7.right = n6

In [60]:
node_list = [n7]

while len(node_list) > 0:
    n = node_list[0]
    print(n.value, n.code)
    node_list.remove(n)
    if n.left:
        node_list.append(n.left)
    if n.right:
        node_list.append(n.right)

410 None
200 j
210 None
100 k
110 None
95 u
15 v


In [83]:
def dfs(n, huff, c = ''):
    if n.code is not None:
        print(n.code, c)
        huff[n.code] = c
    if n.left:
        dfs(n.left, huff, c + '1')
    if n.right:
        dfs(n.right, huff, c + '0')

In [84]:
dfs(n7)

TypeError: dfs() missing 1 required positional argument: 'huff'

In [85]:
from collections import Counter

In [86]:
node_list = []

for key, value in Counter(fd).most_common():
    n = node()
    n.code = key
    n.value = value
    node_list.append(n)

In [87]:
while len(node_list) > 1:
    node_list.sort()
    n1 = node_list[0]
    n2 = node_list[1]
    n3 = node()
    n3.value = n1.value + n2.value
    n3.left = n1
    n3.right = n2
    node_list.remove(n1)
    node_list.remove(n2)
    node_list.append(n3)

In [88]:
huff = {}

dfs(node_list[0], huff)

255 11111111
0 11111110
155 11111101
120 11111100
12 11111011
128 11111010
211 11111001
212 11111000
214 11110111
208 11110110
210 11110101
215 11110100
213 11110011
209 11110010
127 11110001
191 11110000
201 11101111
63 11101110
79 11101101
31 11101100
77 11101011
228 11101010
73 11101001
121 11101000
195 11100111
15 11100110
220 11100101
135 11100100
143 11100011
179 11100010
72 11100001
55 11100000
147 11011111
253 11011110
111 11011101
227 11011100
159 11011011
78 11011010
97 11011001
71 11011000
250 11010111
230 11010110
103 11010101
238 11010100
205 11010011
254 11010010
188 11010001
158 11010000
100 11001111
185 11001110
119 11001101
126 11001100
70 11001011
146 11001010
242 11001001
229 11001000
198 11000111
27 11000110
167 11000101
241 11000100
223 11000011
219 11000010
252 11000001
67 11000000
199 10111111
110 10111110
75 10111101
134 10111100
207 10111011
114 10111010
251 10111001
52 10111000
132 10110111
60 10110110
239 10110101
235 10110100
99 10110011
245 10110010
243 101

In [89]:
huff

{255: '11111111',
 0: '11111110',
 155: '11111101',
 120: '11111100',
 12: '11111011',
 128: '11111010',
 211: '11111001',
 212: '11111000',
 214: '11110111',
 208: '11110110',
 210: '11110101',
 215: '11110100',
 213: '11110011',
 209: '11110010',
 127: '11110001',
 191: '11110000',
 201: '11101111',
 63: '11101110',
 79: '11101101',
 31: '11101100',
 77: '11101011',
 228: '11101010',
 73: '11101001',
 121: '11101000',
 195: '11100111',
 15: '11100110',
 220: '11100101',
 135: '11100100',
 143: '11100011',
 179: '11100010',
 72: '11100001',
 55: '11100000',
 147: '11011111',
 253: '11011110',
 111: '11011101',
 227: '11011100',
 159: '11011011',
 78: '11011010',
 97: '11011001',
 71: '11011000',
 250: '11010111',
 230: '11010110',
 103: '11010101',
 238: '11010100',
 205: '11010011',
 254: '11010010',
 188: '11010001',
 158: '11010000',
 100: '11001111',
 185: '11001110',
 119: '11001101',
 126: '11001100',
 70: '11001011',
 146: '11001010',
 242: '11001001',
 229: '11001000',
 198: '

In [90]:
string = ''

for char in fd:
    string += huff[char]

In [91]:
string[:100]

'1111111101000011111111110001101011111110000101101000101111001011111010011100101111111110000000010000'